# DeepPavlov experiments

В этом ноутбуке я разбираюсь в том, как работать с библиотекой DeepPavlov.

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import gc
import sys
import os
sys.path.append('..')

import dotenv
import numpy as np
import pandas as pd

from IPython.display import display

from deeppavlov import build_model, configs
from transformers import BertForMaskedLM, BertTokenizer, BertConfig

from src.models.BertScorer.bert_scorer_sentence import BertScorerSentence

Зададим константы и переменные окружения.

In [5]:
PROJECT_PATH = os.path.join(os.path.abspath(''), os.pardir)
CONFIGS_PATH = os.path.join(PROJECT_PATH, 'src', 'configs')
os.environ['DP_PROJECT_PATH'] = PROJECT_PATH

## Загрузим Kenlm Pruner

Начнем с того, что протестируем, как работает написанный прунер для гипотез. По итогу он должен оставить 100 гипотез, среди которых в будущем должна быть выбрана наилучшая.

In [6]:
CONFIG_PATH = os.path.join(CONFIGS_PATH, 'levenstein_kenlm_selector.json')

In [7]:
model = build_model(CONFIG_PATH, download=True)

[nltk_data] Downloading package punkt to /home/mrgeekman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2020-11-29 19:11:25.798 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/mrgeekman/Documents/MIPT/НИР/Repo/data/external/russian_words/russian_words_vocab.dict]


In [8]:
lines = ['Опофеозом дня для меня сегодня стала фраза услышанная в новостях.']
results = model(lines)[0]  

In [9]:
results[:5]

[(-52.1003520488739,
  ['апофеозом',
   'дня',
   'для',
   'меня',
   'сегодня',
   'стала',
   'фраза',
   'услышанная',
   'в',
   'новостях',
   '.',
   '</s>']),
 (-53.79301333427429,
  ['опофеозом',
   'дня',
   'для',
   'меня',
   'сегодня',
   'стала',
   'фраза',
   'услышанная',
   'в',
   'новостях',
   '.',
   '</s>']),
 (-55.94359278678894,
  ['апофеозом',
   'для',
   'для',
   'меня',
   'сегодня',
   'стала',
   'фраза',
   'услышанная',
   'в',
   'новостях',
   '.',
   '</s>']),
 (-56.38600826263428,
  ['апофеозом',
   'дни',
   'для',
   'меня',
   'сегодня',
   'стала',
   'фраза',
   'услышанная',
   'в',
   'новостях',
   '.',
   '</s>']),
 (-57.25041913986206,
  ['апофеозом',
   'дно',
   'для',
   'меня',
   'сегодня',
   'стала',
   'фраза',
   'услышанная',
   'в',
   'новостях',
   '.',
   '</s>'])]

Как видим, корректное исправление есть в списке гипотез.

## Применение BertScorer

Теперь попробуем применить к полученным кандидатам BertScorer.

In [10]:
BERT_PATH = os.path.join(PROJECT_PATH, 'models', 'rubert')

In [12]:
config = BertConfig.from_json_file(
    os.path.join(BERT_PATH, 'bert_config.json')
)
model = BertForMaskedLM.from_pretrained(
    os.path.join(BERT_PATH, 'pytorch_model.bin'),
    config=config
)
tokenizer = BertTokenizer(os.path.join(BERT_PATH, 'vocab.txt'))

Some weights of the model checkpoint at /home/mrgeekman/Documents/MIPT/НИР/Repo/notebooks/../models/rubert/pytorch_model.bin were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at /home/mrgeekman/Documents/MIPT/НИР/Repo/notebooks/../models/rubert/pytorch_model.bin and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to 

In [13]:
scorer = BertScorerSentence(model, tokenizer)

In [15]:
sentences_for_bert = [(' '.join(x[1][:-1])) for x in results]

In [16]:
bert_results = scorer(sentences_for_bert, batch_size=32)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [17]:
sort_indices = np.argsort(-np.array(bert_results))

In [20]:
for idx in sort_indices[:5]:
    print(f'{sentences_for_bert[idx]} : {bert_results[idx]:.3f}')

апофеозом дня для меня сегодня стала фраза услышанная в новостях . : -3.660
апофеозом дня для меня сего дня стала фраза услышанная в новостях . : -3.695
апофеозом дня для мене сегодня стала фраза услышанная в новостях . : -3.742
апофеозом пня для меня сегодня стала фраза услышанная в новостях . : -3.841
апофеозом дыня для меня сегодня стала фраза услышанная в новостях . : -3.850


Видим вполне вменяемый результат.